Ici on tente de faire un mapping entre les lieux de la db de corenting et l'API gouv

In [37]:
import pandas as pd
import numpy as np
import requests

On crée une liste des lieux à géocoder.
A chaque tentative, on supprime de la liste les éléments trouvés

In [38]:
# recuperaztion des lieux de corenting
session_data = pd.read_csv("../data/raw/session_planning_olympics.csv")
lieu_epreuves = set(session_data.lieu_epreuve)

In [39]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 40


In [40]:
data = pd.DataFrame([])

In [41]:
# méthode directe
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        # pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_basic.csv")
        data = pd.concat([data, pd.DataFrame(res.json()['results'])])
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)

trouvé
trouvé
trouvé


In [42]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 37


In [43]:
# méthode caps
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu.upper()}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        # pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_upper.csv")
        data = pd.concat([data, pd.DataFrame(res.json()['results'])])
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)
print('longueur restantes', len(lieu_epreuves))

trouvé
trouvé
trouvé
trouvé
longueur restantes 33


In [44]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 33


In [45]:
# méthode caps
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu.lower()}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        # pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_lower.csv")
        data = pd.concat([data, pd.DataFrame(res.json()['results'])])
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)

print('longueur restantes', len(lieu_epreuves))

longueur restantes 33


In [46]:
data = data.reset_index(drop=True)

In [47]:
data = data.loc[data.carac72 != 0].reset_index(drop=True)

In [48]:
data

,numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy,carac72
0,I422180156,Stade Geoffroy-Guichard,14 rue Paul et Pierre Guichard,42000,Saint-Étienne,42218,Terrain de football Honneur,4.390171,45.460749,41000.0
1,I590090003,Stade Pierre Mauroy,"261 Boulevard de Tournai, « Borne de l'espoir ...",59491,Villeneuve-d'Ascq,59009,Terrain du stade Pierre Mauroy,3.130631,50.611962,50154.0
2,I590090003,Stade Pierre Mauroy,"261 Boulevard de Tournai, « Borne de l'espoir ...",59491,Villeneuve-d'Ascq,59009,Arena,3.130631,50.611962,31560.0
3,I610060024,Centre aquatique,31 r du paty,61200,Argentan,61006,Bassin sportif,-0.013203,48.741568,150.0
4,I751160053,PARC DES PRINCES,24 RUE DU COMMANDANT GUILBAUD,75016,Paris 16e Arrondissement,75116,TERRAIN D'HONNEUR,2.253050,48.841450,45500.0
5,I441090221,STADE DE LA BEAUJOIRE,Avenue de la Beaujoire,44000,Nantes,44109,stade Louis Fonteneau,-1.524660,47.256100,37427.0
6,I441090066,GRAND PALAIS,route de saint Joseph parc des expositions,44000,Nantes,44109,grand palais,-1.533060,47.258550,1500.0
7,I930660048,STADE DE FRANCE,Avenue du Stade de France,93200,Saint-Denis,93066,TERRAIN DE FOOTBALL,2.360100,48.924450,80000.0
8,I930660048,STADE DE FRANCE,Avenue du Stade de France,93200,Saint-Denis,93066,STADE D'ATHLETISME,2.360500,48.925100,75000.0


In [49]:
data.to_csv('../data/raw/mapped_data.csv')